In [7]:
import os
import sys
sys.path.append("..")
BaselineSet = os.listdir('../Baselines/')
print("Baseline Set: ", BaselineSet)
TargetBaseline = BaselineSet[0]
print("Target Baseline: ", TargetBaseline)

TargetDir = "../Baselines/" + TargetBaseline + "/ECNs_results/"

Baseline Set:  ['NOTEARS', 'DAGGNN']
Target Baseline:  NOTEARS


In [8]:
from Modules.LoadData.load_data_ADNI import get_dataloader
import numpy as np
from utils import invert_dict
CausalityList = []
LabelList = []
switcher = {
    "CN": 1,
    "LMCI": 2,
    "EMCI": 3,
    "MCI": 4,
    "SMC": 5
}
reverse_switcher = invert_dict(switcher)
groups = [2,3]
dataloader = get_dataloader(batch_size = 1,parent=2)
for data, id, group in dataloader:
    if switcher[group[0]] in groups:
        file = TargetDir + id[0] + "_" + group[0] + ".npy"
        causality = np.load(file)
        CausalityList.append(causality)
        LabelList.append(switcher[group[0]])

CausalityAll = np.array(CausalityList)
LabelAll = np.array(LabelList)

In [9]:
from sklearn import svm
import numpy as np
from sklearn.model_selection import cross_validate
CausalityAll_flatten = CausalityAll.reshape((CausalityAll.shape[0],-1))
model = svm.SVC(kernel='linear')
cv_results = cross_validate(model, CausalityAll_flatten, LabelAll, cv=5, return_train_score=True)
# print(cv_results)
print("Baseline: ", TargetBaseline)
print("Groups: ", {reverse_switcher[group] for group in groups})
print("Test Scores: ", cv_results["test_score"], "\n-- Mean:", np.mean(cv_results["test_score"]), "  Var:", np.std(cv_results["test_score"]))
print("-- 95% confidence interval:", np.mean(cv_results["test_score"]),"±", 1.96*np.std(cv_results["test_score"]))


Baseline:  NOTEARS
Groups:  {'LMCI', 'EMCI'}
Test Scores:  [0.75 0.85 0.8  0.8  0.8 ] 
-- Mean: 0.8   Var: 0.031622776601683784
-- 95% confidence interval: 0.8 ± 0.06198064213930022
